<a href="https://colab.research.google.com/github/paulcodrea/reddit_humor/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# import os
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from time import time
# from keras.models import Sequential, load_model
# from keras.layers.core import Dense, Dropout 
# from keras.layers import LSTM
# from time import time
# from keras.callbacks import EarlyStopping
# from sklearn.preprocessing import MinMaxScaler
# from pathlib import Path

# import nltk
# import regex as re
# from collections import defaultdict

from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
# from nltk.stem.snowball import EnglishStemmer

In [24]:
# We might have to change the following

config = {
    "learning_rate": 0.001,
    "epochs": 40, 
    "batch_size": 4,
    "train_p": 0.55,
    "val_p": 0.05,
    "LSTM_layer": [50, 100],
    "Dropout_layer": [0.15, 0.2],
    "activation": 'tanh',
    "timesteps": 1,
}

In [67]:
class LSTM_model:
    def __init__(self, path):
        self.path = path # Path to the dataset
        self.data = pd.DataFrame() # Dataframe to store the dataset

        self.context_window = 3 # Context window size
        self.w2v_feature_vector = []

    def read_dataset(self):
        """
        Reads the dataset from the given path.
        """
        ret = pd.read_csv(self.path)
        ret.drop(columns=['Unnamed: 0'], inplace=True)
        
        return ret

    def preprocess_text(self):
        """
        Preprocesses the text data.
        """
        self.data['tokens'] = self.data['joke'].apply(word_tokenize) # tokenize the text but keep the punctuation

    # get the maximum size of tokens in the dataset and add to column
    def get_max_tokens(self):
        self.data['max_tokens'] = self.data['tokens'].apply(lambda x: len(x))


    def construct_word2vec(self, max_length):
        """
        Constructs the word2vec model. (Feature vector)
        """
        self.w2v_feature_vector = []
        context_words = [] # Construct window list for word2vec
        
        for line in self.data['tokens']:
            for index, word in enumerate(line):
                if self.context_window > 0:
                    left = index - self.context_window//2
                    right = index + self.context_window//2 + 1
                else:
                    left = index - self.context_window//2
                    right = index + self.context_window//2
                context_words.append([line[i] for i in range(left, right) if i >= 0 and i < len(line)])
 

        # Create a word2vec model
        # context_words = [['a', 'b'], ['a', 'b', 'c'], ['b', 'c', 'd'], ['c', 'd', 'e'], ['d', 'e']] -> list of lists of words and window size is 5
        # vector_size = 50 -> dimension of the feature vector (pairs)
        # min_count = 4 -> minimum number of occurrences of a word in the corpus
        # workers = 4 -> number of threads to use
        # window = 5 -> window size
        model = Word2Vec(context_words, vector_size=max_length, window=self.context_window, workers=4)

        for line in self.data['tokens']:
            for index, word in enumerate(line):
                if word in model.wv.key_to_index:
                    self.w2v_feature_vector.append(model.wv.get_vector(word))
                else:
                    # if the word is not in the model, then add zero. 
                    self.w2v_feature_vector.append(np.zeros(max_length))


In [69]:
# SETTINGS for local machine - change this for Goolg Colab
path = "dataset/final_jokes(1283).csv" #"/content/drive/MyDrive/NLU_Humor-detection/final_jokes(1283).csv"

joke_model = LSTM_model(path)
joke_model.data = joke_model.read_dataset()
joke_model.preprocess_text()
joke_model.get_max_tokens() # get the maximum number of tokens. Since we need the word2vec feature vector to be of the same size for all jokes. 
max_length_joke = joke_model.data['max_tokens'].max()
print(max_length_joke)
joke_model.construct_word2vec(max_length_joke)

# print(joke_model.data['tokens'][0])
# print(joke_model.w2v_feature_vector[0])

405
['My', '5', 'year', 'old', 'just', 'ran', 'out', 'of', 'her', 'room', 'to', 'tell', 'me', 'this', 'joke', 'she', 'just', 'thought', 'up', ':', 'what', 'did', 'the', 'cow', 'say', 'after', 'he', 'was', 'fed', '?', 'Moooooooooore', '!', 'I', "'ve", 'never', 'been', 'this', 'proud', 'of', 'anything', 'in', 'my', 'life', '.']
[ 1.13410121e-02  6.36864156e-02  1.31977141e-01 -8.45313370e-02
  1.94486275e-01  1.29819727e-02  1.71384156e-01  3.88608612e-02
 -1.78204372e-03  1.57712653e-04 -9.69445740e-04 -5.90142347e-02
  1.83116078e-01  5.68029173e-02 -1.81474730e-01 -1.52510896e-01
  1.06074832e-01 -8.47889483e-02 -3.03933471e-02 -2.81293634e-02
 -1.08317055e-01  3.90684977e-02  1.58262193e-01  5.51112220e-02
  5.53292222e-02  4.84247953e-02 -1.11941822e-01 -1.67272300e-01
 -1.84663348e-02 -5.32001816e-02 -8.83795843e-02  1.90830929e-03
  4.14039418e-02 -1.38967395e-01  8.77025723e-03  9.60527137e-02
 -1.96031705e-02 -1.25624269e-01 -5.74848838e-02 -1.25437498e-01
 -4.73934412e-02 -2.45